In [1]:
from lib2to3.pgen2 import driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd 
import time
import concurrent.futures
from IPython.display import display

In [2]:
CHROME_PATH = 'C:/Program Files/Google/Chrome/Application/chrome.exe.'
DRIVER_PATH = 'chromedriver.exe'
WINDOW_SIZE = "1920,1200"

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=%s" % WINDOW_SIZE)
options.binary_location = CHROME_PATH


In [3]:
t1 = time.perf_counter()
l_id = []
data = pd.read_csv('testdata.csv')
m_id = list(data['ID'])
for i in range(0,len(m_id),10):
    l_id.append(m_id[i:i+10])

In [11]:
def work(l_id):
    def web(id):
        driver.get('https://www.imdb.com/title/'+ id)

        try:
            m_name = driver.find_element_by_xpath('//*[@class="sc-b73cd867-0 eKrKux"]').text
        except:
            m_name = 'N/A'

        try:
            rating = driver.find_element_by_xpath('//*[@class="sc-7ab21ed2-1 jGRxWM"]').text
            rating = float(rating)
        except:
            rating = 'N/A'

        try:
            year = driver.find_element_by_xpath('/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/div/ul/li[1]/a').text
        except:
            year = 'N/A'

        try:
            director = driver.find_element_by_xpath('//*[@class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]').text
            if director == '':
                director = 'N/A'
        except:
            director = 'N/A'

        gen = driver.find_elements_by_xpath('//*[@class="ipc-inline-list ipc-inline-list--show-dividers baseAlt"]')
        if len(gen) > 0:
            genres = []
            for i in gen:
                genres.append(i.text)
        else:
            genres = 'N/A'

        ac = driver.find_elements_by_xpath('//ul[@class="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt"][2]/li')
        if len(ac) > 0:
            actors = []
            for i in ac:
                actors.append(i.text)
        else:
            actors = 'N/A'

        new_row = dict({'ID':id,
                        'name': m_name,
                        'rating': rating,
                        'year': year,
                        'director': director,
                        'actors': actors,
                        'genres': genres,
                        })
        return new_row

    df = pd.DataFrame()
    driver = webdriver.Chrome(executable_path = DRIVER_PATH, chrome_options = options)
    for id in l_id:
        df = df.append(web(id), ignore_index=True)
    driver.close()
    df.to_csv('contextsheet.csv', mode='a', header=False, index=False) 

In [24]:
mainsheet = pd.DataFrame(columns=['ID','name', 'rating', 'year', 'director', 'actors', 'genres'])
mainsheet.to_csv('contextsheet.csv',index=False)
with concurrent.futures.ThreadPoolExecutor(max_workers = 6) as executor:
    executor.map(work, l_id)
# work(l_id[0])

C:\Users\debar\AppData\Local\Temp\ipykernel_23488\2310603273.py:55: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = DRIVER_PATH, chrome_options = options)
C:\Users\debar\AppData\Local\Temp\ipykernel_23488\2310603273.py:55: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = DRIVER_PATH, chrome_options = options)
C:\Users\debar\AppData\Local\Temp\ipykernel_23488\2310603273.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  m_name = driver.find_element_by_xpath('//*[@class="sc-b73cd867-0 eKrKux"]').text
C:\Users\debar\AppData\Local\Temp\ipykernel_23488\2310603273.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rating = driver.find_element_by_xpath('//*[@class="sc-7ab21ed2-1 jGRxWM"]').text
C:\Users\debar\A

: 

In [23]:
# df = work(l_id)
display(pd.read_csv('contextsheet.csv'))

,ID,name,rating,year,director,actors,genres
0,tt6517874,Ramante Edenthottam,5.9,2017.0,Ranjith Sankar,NaN,"['Comedy', 'Drama', 'Romance']"
1,tt0150519,Faslah,5.6,1974.0,Khwaja Ahmad Abbas,NaN,"['Crime', 'Drama', 'Romance']"
2,tt4820566,Jawani Ki Ghabrahat,NaN,1991.0,Thriller,NaN,['Thriller']
3,tt0318296,Anbu Chinnam,NaN,1990.0,Add content advisory,NaN,NaN
4,tt12717866,Naveed,NaN,2020.0,Michael Keenan,NaN,['Documentary']
...,...,...,...,...,...,...,...
81,tt13088566,The Rubiks game,NaN,2014.0,Abhilash Mathew,NaN,"['Short', 'Thriller']"
82,tt9053354,Anthervedam,5.8,2018.0,Ravi Kishore Chandina,NaN,['Thriller']
83,tt13605848,Enakkul Oru Kadhal,NaN,2009.0,M. Booman,NaN,['Drama']
84,tt1283527,Samsara Nauka,NaN,1989.0,D. Rajendra Babu,NaN,NaN


In [7]:
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

Finished in 41.5270011 seconds
